In [1]:
import os
import requests
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
from io import BytesIO
from time import sleep
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import shutil

In [2]:
# Diretório base
base_url = "https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/"

In [3]:
# Função para obter o diretório mais recente que já foi baixado
def get_last_downloaded_dir(file_path):
    """
    Lê o nome do diretório mais recente salvo em um arquivo .txt.
    """
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return f.read().strip()
    return None

# Função para salvar o nome do diretório mais recente
def save_last_downloaded_dir(directory_name, file_path):
    """
    Salva o nome do diretório mais recente em um arquivo .txt.
    """
    with open(file_path, 'w') as f:
        f.write(directory_name)

# Função para obter o diretório mais recente
def get_latest_directory(base_url):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    dirs = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('/')]
    latest_dir = sorted(dirs)[-1]  # Obtém o diretório mais recente
    return base_url + latest_dir

# Função para configurar uma sessão com retry
def requests_retry_session(retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504), session=None):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

# Função para baixar e extrair os arquivos
def download_files(directory_url, download_dir, txt_file):
    """
    Baixa os arquivos zip do diretório mais recente e limpa o diretório anterior se houver um novo.
    """
    # Verifica o último diretório baixado
    last_downloaded_dir = get_last_downloaded_dir(txt_file)

    if directory_url == last_downloaded_dir:
        print(f"O diretório {directory_url} já foi processado anteriormente. Abortando download.")
        return
    else:
        # Limpa o diretório downloads, caso haja um novo diretório para baixar
        if os.path.exists(download_dir):
            print(f"Limpando a pasta {download_dir}...")
            shutil.rmtree(download_dir)  # Apaga o diretório inteiro
        os.makedirs(download_dir)

    # Caso seja um novo diretório, prossegue com o download
    response = requests_retry_session().get(directory_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    files = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.zip')]

    for file_name in files:
        file_url = directory_url + file_name
        local_path = os.path.join(download_dir, file_name)

        print(f"Baixando {file_name} para {local_path}...")

        for attempt in range(5):  # Tentar até 5 vezes
            try:
                r = requests_retry_session().get(file_url, stream=True)
                r.raise_for_status()  # Verifica se o download foi bem-sucedido

                with open(local_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                print(f"{file_name} baixado com sucesso.")
                break  # Sai do loop se o download for bem-sucedido

            except (requests.exceptions.RequestException) as e:
                print(f"Erro ao baixar {file_name}: {e}")
                if attempt < 4:
                    print("Tentando novamente...")
                    sleep(5)
                else:
                    print(f"Falha ao baixar {file_name} após {attempt+1} tentativas.")

    # Atualiza o arquivo com o nome do diretório mais recente após baixar todos os arquivos
    save_last_downloaded_dir(directory_url, txt_file)
    print(f"Nome do diretório salvo: {directory_url}")


In [4]:
# Execução
if __name__ == "__main__":
    download_dir = './downloads/'
    txt_file = 'last_downloaded_dir.txt'
    latest_directory = get_latest_directory(base_url)

    download_files(latest_directory, download_dir, txt_file)


Limpando a pasta ./downloads/...
Baixando Cnaes.zip para ./downloads/Cnaes.zip...
Cnaes.zip baixado com sucesso.
Baixando Empresas0.zip para ./downloads/Empresas0.zip...
Empresas0.zip baixado com sucesso.
Baixando Empresas1.zip para ./downloads/Empresas1.zip...
Empresas1.zip baixado com sucesso.
Baixando Empresas2.zip para ./downloads/Empresas2.zip...
Empresas2.zip baixado com sucesso.
Baixando Empresas3.zip para ./downloads/Empresas3.zip...
Empresas3.zip baixado com sucesso.
Baixando Empresas4.zip para ./downloads/Empresas4.zip...
Empresas4.zip baixado com sucesso.
Baixando Empresas5.zip para ./downloads/Empresas5.zip...
Empresas5.zip baixado com sucesso.
Baixando Empresas6.zip para ./downloads/Empresas6.zip...
Empresas6.zip baixado com sucesso.
Baixando Empresas7.zip para ./downloads/Empresas7.zip...
Empresas7.zip baixado com sucesso.
Baixando Empresas8.zip para ./downloads/Empresas8.zip...
Empresas8.zip baixado com sucesso.
Baixando Empresas9.zip para ./downloads/Empresas9.zip...
Em